Treat image ROI takes electron diffraction patterns and calibrate the image, rotate it, draw the BZ around the Bragg peak and average equivalent BZ together

In [1]:
import sys
sys.path.append('../../')
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import gridspec
from matplotlib.patches import Polygon
from matplotlib.patches import Circle
from mpl_toolkits.axes_grid1 import make_axes_locatable
import imutils
import glob
import cv2  # importing cv
import find_peak
import scipy.special as special
from matplotlib import ticker
from scipy.fft import fft, fftfreq
from scipy.optimize import curve_fit
import nfft
import scipy
import scipy.fft
import functions_analysis as fa
from skimage import img_as_float
from skimage import filters
%matplotlib inline 

path = r'C:\Users\rclaude\Documents\projet\graphite\drawing\UED_fig'
path_main=path+r'\data'
scans = [ '1.55 eV',  '3.1 eV' ]
scan_number = [ 'r464',  'r471' ]

data = {}

for k, scan_name in enumerate(scans):
    data[scan_name] = {}
    data[scan_name] = {}
    df= pd.read_pickle(path_main + r'\PROC_'+ scan_number[k], 
                       compression={'method': 'gzip', 'compresslevel': 1, 'mtime': 1})

    data[scan_name]['imgOFF'] = np.stack(df['imgOFF'])
    data[scan_name]['imgON'] = np.stack(df['imgON'])
    data[scan_name]['delay'] = np.stack(df['delay'])
    data[scan_name]['t0_index'] = df.attrs['t0_index']
    # print(df.attrs['peak_dist'])
    data[scan_name]['Zorder'] = df.attrs['Zorder_pos']
    data[scan_name]['cal'] = df.attrs['calibration']

    data[scan_name]['kx'] = data[scan_name]['cal'] *(np.arange(np.size(data[scan_name]['imgON'][0],1)) -
                                                      data[scan_name]['Zorder'] [1])
    data[scan_name]['ky'] = data[scan_name]['cal'] *(np.arange(np.size(data[scan_name]['imgON'][0],0)) - 
                                                     data[scan_name]['Zorder'] [0])


In [ ]:
L = df.attrs['peak_dist']
Zorder = df.attrs['Zorder_pos']
cal = df.attrs['calibration']
t0 = df.attrs['t0_index']

# cc = L2/2 * np.exp(1j * np.pi * ( 1./2 * np.arange(5)))## for squared BZ
cc =  L * np.sqrt(3)/3 *np.exp(1j * np.pi  * ( 1./3 * np.arange(7))) ## for hexagonal BZ


#### construction position of the Bragg peak, ordered by family
BP_BZ = []
for i in range(6):
    BP_BZ.append([L*np.cos(np.pi*i/3), L*np.sin(np.pi*i/3)])
BP_pos = [Zorder + np.array(BP_BZ)]
BP_BZ = []
for i in range(6):
    BP_BZ.append([2*L*np.cos(np.pi*i/3), 2*L*np.sin(np.pi*i/3)])
BP_pos.append(Zorder + np.array(BP_BZ))
BP_BZ = []
for i in range(6):
    BP_BZ.append([np.sqrt(3)*L*np.cos(np.pi*i/3+np.pi/6), np.sqrt(3)*L*np.sin(np.pi*i/3+np.pi/6)])
BP_pos.append(Zorder + np.array(BP_BZ))
BP_name = ['{100}', '{200}', '{110}']
BP_BZ = []
print(BP_pos)


#### construction position of the K-pont, ordered by family

edge = L*np.tan(np.pi/6)
BP_DS = []
for i in range(6):
    BP_DS.append([2*edge*np.cos(np.pi*i/3+np.pi/6), 2*edge*np.sin(np.pi*i/3+np.pi/6)])
DS_pos = [Zorder + np.array(BP_DS)]
BP_DS = []
for i in range(6):
    BP_DS.append([4*edge*np.cos(np.pi*i/3+np.pi/6), 4*edge*np.sin(np.pi*i/3+np.pi/6)])
DS_pos.append(Zorder + np.array(BP_DS))
BP_DS = []
for i in range(6):  
    BP_DS.append(fa.rotate_vector(np.array([3*L/2, edge/2]), i*np.pi/3))
    BP_DS.append(fa.rotate_vector(np.array([3*L/2,-edge/2]), i*np.pi/3))
DS_pos.append(Zorder + np.array(BP_DS))
BP_DS = []
for i in range(6):
    BP_DS.append(fa.rotate_vector(np.array([2*L, edge]), i*np.pi/3))
    BP_DS.append(fa.rotate_vector(np.array([2*L,-edge]), i*np.pi/3))
DS_pos.append(Zorder + np.array(BP_DS))
BP_DS = []
    # BP_DS.append(fa.rotate_vector(np.array([5*L/2, edge/2]), i*np.pi/3))
    # BP_DS.append(fa.rotate_vector(np.array([5*L/2,-edge/2]), i*np.pi/3))

DS_name = [r'$\Gamma \to K$ first order', r'$\Gamma \to K$ second order', r'$\Gamma \to M$ first order', r'$\Gamma \to M$ second order']

In [ ]:
plt.figure(figsize = (16, 8))

lim=15
for k, scan_name in enumerate(scans):

    beforet0 =  data[scan_name]['delay']<-0.5
    # aftert0 = ( data[scan_name]['delay']>0.1 ) & ( data[scan_name]['delay']<2)
    # aftert0 = ( data[scan_name]['delay']>2 ) & ( data[scan_name]['delay']<5)
    aftert0 = ( data[scan_name]['delay']>5 ) & ( data[scan_name]['delay']<10)

    diff=(np.mean(data[scan_name]['imgON'][aftert0], axis=0))/np.mean(data[scan_name]['imgOFF'][aftert0], axis=0) - \
        (np.mean(data[scan_name]['imgON'][beforet0], axis=0))/np.mean(data[scan_name]['imgOFF'][beforet0], axis=0)
    ax = plt.subplot(1,2 ,k+1)
    ax.pcolorfast(data[scan_name]['kx'], data[scan_name]['ky'], diff, vmin=-0.01, vmax = 0.01, cmap = 'bwr')
    plt.gca().set_aspect('equal')

    plt.title(scan_name, pad = 20)

    plt.xlim([-lim, lim]), plt.ylim([-lim, lim])
    plt.xticks([-5, 0, 5]), plt.yticks([-5, 0, 5])

    plt.xlabel(r'Momentum (1/\AA)')
    if k ==0:
        plt.ylabel(r'Momentum (1/\AA)')
    else:
        ax.set_yticklabels([])
    ax.tick_params(axis='both', direction='in', pad = 15)
    for i in range(len(BP_pos)):
        for j in range(len(BP_pos[i])):
            ax.add_patch(Circle((cal*(Zorder[1]-BP_pos[i][j][1]), cal*(Zorder[0]-BP_pos[i][j][0])), 0.1, fill = False, edgecolor = 'k', lw = 0.5))
            ax.add_patch(Polygon(cal*np.c_[(cc.real+Zorder[1]-BP_pos[i][j][1]), (cc.imag+Zorder[0]-BP_pos[i][j][0])],
                lw=0.2, edgecolor='k',
                clip_on=False, facecolor='none'))
            
    

In [4]:
plt.rcParams.update({'font.size': 20})
plt.rc('axes', linewidth=2)
plt.rcParams['xtick.major.size'] = 6
plt.rcParams['xtick.major.width'] = 2
plt.rcParams['xtick.minor.size'] = 3
plt.rcParams['xtick.minor.width'] = 1
plt.rcParams['ytick.major.size'] = 6
plt.rcParams['ytick.major.width'] = 2
plt.rcParams['ytick.minor.size'] = 3
plt.rcParams['ytick.minor.width'] = 1
plt.rc('axes', linewidth=2)
# plt.rc('font', weight='semibold')
# plt.rcParams['text.latex.preamble'] = r'\usepackage{sfmath} \boldmath'
plt.rc('font', family='arial',size=26)#, weight='bold')
plt.rc('text', usetex=False)

In [ ]:
print(scan_name)

In [ ]:
plt.figure(figsize = (16, 8))

lim=7.5
scan_name = '1.55 eV'
aftert0 = ( data[scan_name]['delay']>0 ) & ( data[scan_name]['delay']<8)
ax = plt.subplot(1,2 ,1)
ax.pcolorfast(data[scan_name]['kx'], data[scan_name]['ky'], np.mean(data[scan_name]['imgON'][aftert0], axis=0), vmin=0, vmax = 5e5, cmap = 'plasma')
plt.gca().set_aspect('equal')
plt.title(r'$I_{\text{on}}$', pad = 20)
plt.xlim([-lim, lim]), plt.ylim([-lim, lim])
plt.xticks([]), plt.yticks([])

for i in range(len(BP_pos)):
    for j in range(len(BP_pos[i])):
        # ax.add_patch(Circle((cal*(Zorder[1]-BP_pos[i][j][1]), cal*(Zorder[0]-BP_pos[i][j][0])), 0.1, fill = False, edgecolor = 'k', lw = 0.5))
        ax.add_patch(Polygon(cal*np.c_[(cc.real+Zorder[1]-BP_pos[i][j][1]), (cc.imag+Zorder[0]-BP_pos[i][j][0])],
            lw=0.2, edgecolor='grey', clip_on=False, facecolor='none'))

# ax.add_patch(Polygon(cal*np.c_[(cc.real), (cc.imag)],
#             lw=0.2, edgecolor='grey', clip_on=True, facecolor='white'))
ax = plt.subplot(1,2 ,2)
ax.pcolorfast(data[scan_name]['kx'], data[scan_name]['ky'], np.mean(data[scan_name]['imgOFF'][aftert0], axis=0), vmin=0, vmax = 5e5, cmap = 'plasma')
plt.gca().set_aspect('equal')
plt.title(r'$I_{\text{off}}$', pad = 20)

plt.xlim([-lim, lim]), plt.ylim([-lim, lim])
plt.xticks([]), plt.yticks([])

for i in range(len(BP_pos)):
    for j in range(len(BP_pos[i])):
        # ax.add_patch(Circle((cal*(Zorder[1]-BP_pos[i][j][1]), cal*(Zorder[0]-BP_pos[i][j][0])), 0.1, fill = False, edgecolor = 'k', lw = 0.5))
        ax.add_patch(Polygon(cal*np.c_[(cc.real+Zorder[1]-BP_pos[i][j][1]), (cc.imag+Zorder[0]-BP_pos[i][j][0])],
            lw=0.2, edgecolor='grey', clip_on=False, facecolor='none'))
    
# ax.add_patch(Polygon(cal*np.c_[(cc.real), (cc.imag)],
#             lw=0.2, edgecolor='grey', clip_on=True, facecolor='white'))
plt.savefig('.\diffraction_ON_OFF.svg', format='svg')
        

In [7]:
for scan_name in scans:
    data[scan_name]['imgON_not_filtered'] = data[scan_name]['imgON']
    data[scan_name]['imgOFF_not_filtered'] = data[scan_name]['imgOFF']

In [8]:
s = 1
for scan_name in scans:
    data[scan_name]['imgON'] = filters.gaussian(img_as_float(data[scan_name]['imgON']), sigma=s)
    data[scan_name]['imgOFF'] = filters.gaussian(img_as_float(data[scan_name]['imgOFF']), sigma=s)

In [ ]:
plt.figure(figsize = (16, 8))



lim=7.5
scan_name = '1.55 eV'
aftert0 = ( data[scan_name]['delay']>0. ) & ( data[scan_name]['delay']<2.5)
aftert0 = ( data[scan_name]['delay']>2.5) & ( data[scan_name]['delay']<6)
aftert0 = ( data[scan_name]['delay']>6) & ( data[scan_name]['delay']<10)


beforet0 =  (data[scan_name]['delay'] < -0.2)

diff0 = (data[scan_name]['imgON'][1])/data[scan_name]['imgOFF'][1] - \
            (np.mean(data[scan_name]['imgON'][beforet0], axis=0))/np.mean(data[scan_name]['imgOFF'][beforet0], axis=0)

diff1 = (np.mean(data[scan_name]['imgON'][aftert0], axis=0))/np.mean(data[scan_name]['imgOFF'][aftert0], axis=0) - \
            (np.mean(data[scan_name]['imgON'][beforet0], axis=0))/np.mean(data[scan_name]['imgOFF'][beforet0], axis=0)
sens = 0.02

ax = plt.subplot(1,2 ,1)
ax.pcolorfast(data[scan_name]['kx'], data[scan_name]['ky'], diff0, vmin=-sens, vmax = sens, cmap = 'seismic')
plt.gca().set_aspect('equal')
# plt.title(r'before t0', pad = 20)
plt.xlim([-lim, lim]), plt.ylim([-lim, lim])
plt.xticks([]), plt.yticks([])

for i in range(len(BP_pos)):
    for j in range(len(BP_pos[i])):
        # ax.add_patch(Circle((cal*(Zorder[1]-BP_pos[i][j][1]), cal*(Zorder[0]-BP_pos[i][j][0])), 0.1, fill = False, edgecolor = 'k', lw = 0.5))
        ax.add_patch(Polygon(cal*np.c_[(cc.real+Zorder[1]-BP_pos[i][j][1]), (cc.imag+Zorder[0]-BP_pos[i][j][0])],
            lw=0.2, edgecolor='grey', clip_on=False, facecolor='none'))

ax.add_patch(Polygon(cal*np.c_[(cc.real), (cc.imag)],
            lw=0.2, edgecolor='grey', clip_on=True, facecolor='white'))


ax = plt.subplot(1,2 ,2)
ax.pcolorfast(data[scan_name]['kx'], data[scan_name]['ky'], diff1, vmin=-sens, vmax = sens, cmap = 'seismic')
plt.gca().set_aspect('equal')
# plt.title(r'after t0', pad = 20)

plt.xlim([-lim, lim]), plt.ylim([-lim, lim])
plt.xticks([]), plt.yticks([])

for i in range(len(BP_pos)):
    for j in range(len(BP_pos[i])):
        # ax.add_patch(Circle((cal*(Zorder[1]-BP_pos[i][j][1]), cal*(Zorder[0]-BP_pos[i][j][0])), 0.1, fill = False, edgecolor = 'k', lw = 0.5))
        ax.add_patch(Polygon(cal*np.c_[(cc.real+Zorder[1]-BP_pos[i][j][1]), (cc.imag+Zorder[0]-BP_pos[i][j][0])],
            lw=0.2, edgecolor='grey', clip_on=False, facecolor='none'))
        
ax.add_patch(Polygon(cal*np.c_[(cc.real), (cc.imag)],
            lw=0.2, edgecolor='grey', clip_on=True, facecolor='white'))
        
plt.savefig('.\diffraction_normalized_800nm.svg', format='svg')
        

In [ ]:
plt.figure(figsize = (27, 9))



lim=7.5
scan_name = '1.55 eV'
aftert0 = ( data[scan_name]['delay']>0.8 ) & ( data[scan_name]['delay']<2.5)
aftert0 = ( data[scan_name]['delay']>2.5) & ( data[scan_name]['delay']<6)
aftert0 = ( data[scan_name]['delay']>6) & ( data[scan_name]['delay']<10)


beforet0 =  (data[scan_name]['delay'] < -0.2)



diff1 = (np.mean(data[scan_name]['imgON'][aftert0], axis=0))/np.mean(data[scan_name]['imgOFF'][aftert0], axis=0) - \
            (np.mean(data[scan_name]['imgON'][beforet0], axis=0))/np.mean(data[scan_name]['imgOFF'][beforet0], axis=0)
sens = 0.02

ax = plt.subplot(2,4 ,1)
scan_name = '1.55 eV'
diff0 = (data[scan_name]['imgON'][1])/data[scan_name]['imgOFF'][1] - \
            (np.mean(data[scan_name]['imgON'][beforet0], axis=0))/np.mean(data[scan_name]['imgOFF'][beforet0], axis=0)
ax.pcolorfast(data[scan_name]['kx'], data[scan_name]['ky'], diff0, vmin=-sens, vmax = sens, cmap = 'seismic')
plt.gca().set_aspect('equal')
# plt.title(r'before t0', pad = 20)
plt.xlim([-lim, lim]), plt.ylim([-lim, lim])
plt.xticks([]), plt.yticks([])

ax = plt.subplot(2,4 ,5)
scan_name = '1.55 eV'
diff0 = (data[scan_name]['imgON'][1])/data[scan_name]['imgOFF'][1] - \
            (np.mean(data[scan_name]['imgON'][beforet0], axis=0))/np.mean(data[scan_name]['imgOFF'][beforet0], axis=0)
ax.pcolorfast(data[scan_name]['kx'], data[scan_name]['ky'], diff0, vmin=-sens, vmax = sens, cmap = 'seismic')
plt.gca().set_aspect('equal')
# plt.title(r'before t0', pad = 20)
plt.xlim([-lim, lim]), plt.ylim([-lim, lim])
plt.xticks([]), plt.yticks([])

for i in range(len(BP_pos)):
    for j in range(len(BP_pos[i])):
        # ax.add_patch(Circle((cal*(Zorder[1]-BP_pos[i][j][1]), cal*(Zorder[0]-BP_pos[i][j][0])), 0.1, fill = False, edgecolor = 'k', lw = 0.5))
        ax.add_patch(Polygon(cal*np.c_[(cc.real+Zorder[1]-BP_pos[i][j][1]), (cc.imag+Zorder[0]-BP_pos[i][j][0])],
            lw=0.2, edgecolor='grey', clip_on=False, facecolor='none'))

ax.add_patch(Polygon(cal*np.c_[(cc.real), (cc.imag)],
            lw=0.2, edgecolor='grey', clip_on=True, facecolor='white'))


ax = plt.subplot(1,2 ,2)
ax.pcolorfast(data[scan_name]['kx'], data[scan_name]['ky'], diff1, vmin=-sens, vmax = sens, cmap = 'seismic')
plt.gca().set_aspect('equal')
# plt.title(r'after t0', pad = 20)

plt.xlim([-lim, lim]), plt.ylim([-lim, lim])
plt.xticks([]), plt.yticks([])

for i in range(len(BP_pos)):
    for j in range(len(BP_pos[i])):
        # ax.add_patch(Circle((cal*(Zorder[1]-BP_pos[i][j][1]), cal*(Zorder[0]-BP_pos[i][j][0])), 0.1, fill = False, edgecolor = 'k', lw = 0.5))
        ax.add_patch(Polygon(cal*np.c_[(cc.real+Zorder[1]-BP_pos[i][j][1]), (cc.imag+Zorder[0]-BP_pos[i][j][0])],
            lw=0.2, edgecolor='grey', clip_on=False, facecolor='none'))
        
ax.add_patch(Polygon(cal*np.c_[(cc.real), (cc.imag)],
            lw=0.2, edgecolor='grey', clip_on=True, facecolor='white'))
        
plt.savefig('.\diffraction_normalized_800nm.svg', format='svg')

In [ ]:

a= 2.46
# a = 1.42
b =(4*np.pi/(np.sqrt(3)*a)) 
print(b)
cal_b = cal/b
print(cal_b)